In [1]:
import numpy as np 
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import RobustScaler

from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d

# With sampling

In [2]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666)

In [4]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [6]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

## Loading Encoder

In [8]:
VAE = torch.load("models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300.pt")

In [17]:
VAE

DeepLSTM_VAE(
  (encoder_reshape): Reshape()
  (encoder_lstm): LSTM(1, 128)
  (encoder_output): VariationalLayer(
    (mu): Linear(in_features=128, out_features=15, bias=True)
    (rho): Linear(in_features=128, out_features=15, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
  (decoder_hidden): Linear(in_features=15, out_features=128, bias=True)
  (decoder_lstm): LSTM(1, 128)
  (decoder_output): RecurrentDecoderOutput(
    (mu): Linear(in_features=128, out_features=1, bias=True)
    (rho): Linear(in_features=20480, out_features=1, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
)

## Freezing Encoder

In [10]:
class Encoder_Classifier(nn.Module):
    def __init__(self, encoder, clf):
        super(Encoder_Classifier, self).__init__()
        self.encoder=encoder
        self.clf = clf
        self.frozen_encoder = False

    def freeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        self.frozen_encoder = True
        print("Encoder frozen.")

    def unfreeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = True
        self.frozen_encoder = False
        print("Encoder unfrozen.")
    
    def forward(self, X):
        Z, mu, sigma = self.encoder(X)
        return self.clf(Z)
    

In [39]:
class RNN_Encoder_Classifier(nn.Module):
    def __init__(self, model, clf):
        super(RNN_Encoder_Classifier, self).__init__()
        self.model = model
        self.clf = clf
        self.frozen_encoder = False

    def freeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = False
        self.frozen_encoder = True
        print("Encoder frozen.")

    def unfreeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = True
        self.frozen_encoder = False
        print("Encoder unfrozen.")
    
    def forward(self, X):
        Z, mu, sigma = self.model.encoder(X)
        return self.clf(Z)

In [40]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [41]:
model = RNN_Encoder_Classifier(model=VAE, clf = DNN1)
model.freeze_encoder()

Encoder frozen.


In [42]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-2)

In [43]:
m1 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300",
        verbose=True
        )

cuda:0


In [44]:
lh = m1(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%eps : Float(512, 15) = aten::randn_like(%sigma, %138, %139, %140, %141), scope: RNN_Encoder_Classifier/VariationalLayer # Z:\Semi-supervised-Learning-with-VAE\layers.py:35:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[482, 1] (1.4707341194152832 vs. 1.3906586170196533) and 2045 other locations (99.00%)
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-

Epoch [1/100], average_loss:0.8453, validation_loss:0.6827, val_accuracy:0.7662
Epoch [2/100], average_loss:0.6273, validation_loss:0.6152, val_accuracy:0.7403
Epoch [3/100], average_loss:0.5886, validation_loss:0.5968, val_accuracy:0.7662
Epoch [4/100], average_loss:0.5701, validation_loss:0.5654, val_accuracy:0.8052
Epoch [5/100], average_loss:0.5540, validation_loss:0.5463, val_accuracy:0.7922
Epoch [6/100], average_loss:0.5275, validation_loss:0.5148, val_accuracy:0.7013
Epoch [7/100], average_loss:0.4897, validation_loss:0.4750, val_accuracy:0.8182
Epoch [8/100], average_loss:0.4798, validation_loss:0.4794, val_accuracy:0.7662
Epoch [9/100], average_loss:0.4702, validation_loss:0.4651, val_accuracy:0.7792
Epoch [10/100], average_loss:0.4612, validation_loss:0.4579, val_accuracy:0.9091
Epoch [11/100], average_loss:0.4597, validation_loss:0.4626, val_accuracy:0.7403
Epoch [12/100], average_loss:0.4561, validation_loss:0.4527, val_accuracy:0.8571
Epoch [13/100], average_loss:0.4538, 

In [45]:
X_test = np.load("data/mu_lstm_test.npy")
y_test = np.load("data/labels_test.npy")

In [114]:
#m1.tb.close()

m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model.clf(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [115]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.6910981336881359

In [116]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7047681644359465

In [49]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2343, 1453,   90,  349],
       [2161, 7669,  118,  497],
       [  19,    7,  417,   36],
       [ 158,   46,   10, 1363]], dtype=int64)

In [117]:
torch.save(m1.model.clf, "models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300+DNN_on_VAE_sampling_latent_space_m.IV_lr-{1e-2,1e-3}_bs-512_epoch-100.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sigmoid. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [50]:
lh = m1(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/200], average_loss:0.4226, validation_loss:0.4284, val_accuracy:0.7532
Epoch [102/200], average_loss:0.4217, validation_loss:0.4254, val_accuracy:0.8442
Epoch [103/200], average_loss:0.4236, validation_loss:0.4313, val_accuracy:0.7532
Epoch [104/200], average_loss:0.4221, validation_loss:0.4260, val_accuracy:0.7922
Epoch [105/200], average_loss:0.4213, validation_loss:0.4270, val_accuracy:0.8182
Epoch [106/200], average_loss:0.4221, validation_loss:0.4259, val_accuracy:0.8571
Epoch [107/200], average_loss:0.4221, validation_loss:0.4269, val_accuracy:0.7792
Epoch [108/200], average_loss:0.4225, validation_loss:0.4274, val_accuracy:0.7922
Epoch [109/200], average_loss:0.4219, validation_loss:0.4271, val_accuracy:0.7662
Epoch [110/200], average_loss:0.4223, validation_loss:0.4281, val_accuracy:0.7532
Epoch [111/200], average_loss:0.4220, validation_loss:0.4231, val_accuracy:0.8052
Epoch [112/200], average_loss:0.4206, validation_loss:0.4342, val_accuracy:0.7532
Epoch [113/200],

In [52]:
m1.tb.close()

m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model.clf(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [53]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.6910981336881359

In [54]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7047681644359465

In [55]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2386, 1450,   89,  310],
       [2177, 7651,  127,  490],
       [  28,    5,  413,   33],
       [ 170,   49,   13, 1345]], dtype=int64)

In [ ]:
#torch.save(m1.model.clf, "models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300+DNN_on_VAE_sampling_latent_space_m.IV_lr-{1e-2,1e-3}_bs-512_epoch-200.pt")

# bs 128

In [56]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [105]:
DNN2 = nn.Sequential(
            nn.Linear(in_features=15, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [106]:
model2 = RNN_Encoder_Classifier(VAE, clf = DNN2)
model2.freeze_encoder()

Encoder frozen.


In [107]:
optimizer= torch.optim.Adam(model2.parameters(), lr=1e-2)

In [108]:
m2 = Trainer(
        model=model2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,60], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300",
        verbose=True
        )

cuda:0


In [109]:
lh = m2(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%eps : Float(512, 15) = aten::randn_like(%sigma, %134, %135, %136, %137), scope: RNN_Encoder_Classifier/VariationalLayer # Z:\Semi-supervised-Learning-with-VAE\layers.py:35:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[230, 1] (1.4038562774658203 vs. 2.1659274101257324) and 2047 other locations (100.00%)
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site

Epoch [1/100], average_loss:0.7723, validation_loss:0.6685, val_accuracy:0.7792
Epoch [2/100], average_loss:0.6666, validation_loss:0.6489, val_accuracy:0.8052
Epoch [3/100], average_loss:0.6330, validation_loss:0.6122, val_accuracy:0.8052
Epoch [4/100], average_loss:0.6035, validation_loss:0.5894, val_accuracy:0.8442
Epoch [5/100], average_loss:0.5833, validation_loss:0.5804, val_accuracy:0.8312
Epoch [6/100], average_loss:0.5685, validation_loss:0.5652, val_accuracy:0.7662
Epoch [7/100], average_loss:0.5586, validation_loss:0.5519, val_accuracy:0.7662
Epoch [8/100], average_loss:0.5470, validation_loss:0.5461, val_accuracy:0.8312
Epoch [9/100], average_loss:0.5424, validation_loss:0.5443, val_accuracy:0.7662
Epoch [10/100], average_loss:0.5371, validation_loss:0.5352, val_accuracy:0.8571
Epoch [11/100], average_loss:0.5299, validation_loss:0.5247, val_accuracy:0.7792
Epoch [12/100], average_loss:0.5223, validation_loss:0.5171, val_accuracy:0.7792
Epoch [13/100], average_loss:0.5122, 

In [100]:
lh = m2(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/200], average_loss:0.4233, validation_loss:0.4299, val_accuracy:0.8052
Epoch [102/200], average_loss:0.4242, validation_loss:0.4277, val_accuracy:0.8442
Epoch [103/200], average_loss:0.4237, validation_loss:0.4278, val_accuracy:0.9091
Epoch [104/200], average_loss:0.4230, validation_loss:0.4331, val_accuracy:0.7792
Epoch [105/200], average_loss:0.4247, validation_loss:0.4316, val_accuracy:0.7403
Epoch [106/200], average_loss:0.4239, validation_loss:0.4306, val_accuracy:0.8052
Epoch [107/200], average_loss:0.4232, validation_loss:0.4306, val_accuracy:0.8571
Epoch [108/200], average_loss:0.4236, validation_loss:0.4272, val_accuracy:0.8312
Epoch [109/200], average_loss:0.4234, validation_loss:0.4258, val_accuracy:0.8571
Epoch [110/200], average_loss:0.4245, validation_loss:0.4274, val_accuracy:0.8831
Epoch [111/200], average_loss:0.4236, validation_loss:0.4302, val_accuracy:0.8182
Epoch [112/200], average_loss:0.4240, validation_loss:0.4328, val_accuracy:0.7662
Epoch [113/200],

In [110]:
#m2.tb.close()
m2.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m2.model.clf(torch.tensor(X_test).float().to(m2.device)).cpu().detach(), axis=1)

In [111]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.641180326377472

In [112]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.6717853728489483

In [113]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[1833, 1481,   82,  839],
       [2044, 7565,  130,  706],
       [  11,    4,  391,   73],
       [  69,   46,    8, 1454]], dtype=int64)

In [69]:
m2.tb.close()

In [ ]:
#torch.save(m2.model.clf, "models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300+DNN_(drop)_on_VAE_sampling_latent_space_m.VI_lr-{1e-2,1e-3}_bs-512_epoch-200.pt")